In [1]:
!pip install fastai --upgrade -q
!pip install kornia -q
!pip install pydicom -q

     |████████████████████████████████| 194kB 21.2MB/s 
     |████████████████████████████████| 61kB 7.4MB/s 
     |████████████████████████████████| 225kB 22.1MB/s 
     |████████████████████████████████| 1.9MB 13.5MB/s 


In [1]:
from fastai.vision.all import *
from fastai.vision.widgets import *
import numpy as np
from skimage.morphology import binary_erosion
from skimage.transform import resize
import time

import fastai.losses
import fastai.layers

fastai.layers.BaseLoss = fastai.losses.BaseLoss
fastai.layers.CrossEntropyLossFlat = fastai.losses.CrossEntropyLossFlat
fastai.layers.BCEWithLogitsLossFlat = fastai.losses.BCEWithLogitsLossFlat
fastai.layers.BCELossFlat = fastai.losses.BCELossFlat
fastai.layers.MSELossFlat = fastai.losses.MSELossFlat
fastai.layers.L1LossFlat = fastai.losses.L1LossFlat
fastai.layers.LabelSmoothingCrossEntropy = fastai.losses.LabelSmoothingCrossEntropy
fastai.layers.LabelSmoothingCrossEntropyFlat = fastai.losses.LabelSmoothingCrossEntropyFlat

In [2]:
from fastai.medical.imaging import *
import pydicom

In [3]:
def get_y():
    return 1

In [4]:
def get_learner(cpu=False):
    fname='export.pkl'
    url='https://www.dropbox.com/s/a2rwhy29wx9s448/export_orig.pkl?dl=1'
    # download_url and file_extract are functions from fastai2
    download_url(url,fname)
    learn = load_learner(fname, cpu=cpu)
    os.remove(fname)
    return learn

In [5]:
def plottingWithRedLabel (x,y):
    #Normalising the x data to 0 to 255 
    x=np.array((x-x.min())/(x.max()-x.min())*255,'uint8')
    #Creating the border of the segmentation
    yEroded=binary_erosion(y,np.ones((5,5)))
    yBorder=np.logical_xor(y,yEroded)
    #Clip the red channel value
    xy=np.clip(x+yBorder*255,0,255)
    return np.stack([x,xy,x], axis=-1)

In [6]:
learn = get_learner(cpu=True)

In [7]:
btn_upload = widgets.FileUpload(accept= '.dcm')
out_pl = widgets.Output()
lbl_pred = widgets.Label()

In [8]:
def get_img(fl):
    img = pydicom.dcmread(pydicom.filebase.DicomBytesIO(fl)).pixel_array
    img = tensor(img)
    brks = img.freqhist_bins(n_bins=3500);
    img = PILImage.create(img.hist_scaled(brks)*255)
    return img

In [9]:
def on_click(change):
    out_pl.clear_output()
    img = get_img(btn_upload.data[-1])
    lbl_pred.value = f'Processing...'
    start = time.time()
    p = learn.predict(img)  
    mask = resize(p[1].float().numpy().astype(np.float32), img.shape[:2])
    sugg_contour = plottingWithRedLabel(np.array(img.convert('L')),mask)
    end = time.time()-start
    sugg_contour = Image.fromarray(sugg_contour.astype(np.uint8))

    lbl_pred.value = f'Suggested Contour took {end:01f} seconds to generate'
    
    a = widgets.Output()
    b = widgets.Output()
    with a:
        a.clear_output()
        display(img.convert('L'))
    with b:
        b.clear_output()
        display(sugg_contour)
    
    with out_pl:
        display(HBox([a,b]))
        

In [10]:
btn_upload.observe(on_click, names=['data'])

In [11]:
display(VBox([widgets.Label('Upload a prostate Image:'), btn_upload, out_pl, lbl_pred]))